# Tworzenie zasobów

Algorytmy wykorzystywane w problemach przetwarzania języka naturalnego opierają najczęściej swoje działanie o analizę dużych korpusów danych. O ile w zadaniach konkursowych często odpowiednie dane są już przygotowane, o tyle tworząc własne eksperymenty, często musimy sami pozyskać dane i przetransformować do użytecznej postaci.

Dzisiejsze laboratoria dotyczyć będą tworzenia korpusów danych.

## Automatyczne pozyskiwanie surowych danych tekstowych
Dotychczas omawiane metody działały na surowym tekście, który transformowany był do odpowiedniej reprezentacji wektorowej (Bag of words, bag of ngrams, embeddingi). Jak zautomatyzować pozyskiwanie takich surowych danych z internetu?

W tej części skupimy się na stworzeniu automatycznego pobieracza danych, który działać będzie w dwóch "obszarach":
<ol>
<li>crawler: moduł odwiedzający kolejne strony internetowy</li>
<li>scraper: moduł ekstrahujący treść z konkretnych stron internetowych</li>
</ol>

Wykorzystajmy do tego dwie biblioteki: 

**urllib** - do odwiedzania stron

**BeautifulSoup** - do parsowania danych (np. w formacie HTML).

## Zadanie1: Napisz prosty ekstraktor danych ze stron WWW odwiedzający kilka podstron
Ekstraktor ma odwiedzić zadaną stronę internetową, pobrać zawartość wszystkich tekstów wewnątrz paragrafów (wewnątrz tagów P zawartych w pobranym dokumencie HTML), a następnie odwiedzić 5 dowolnych linków z tej strony i z nich analogicznie pobrać zawartość.
Łącznie powinniśmy otrzymać dane z 6 adresów internetowch (strona główna + 5 linków ze strony głównej).

Do napisania crawlera przydać się mogą następujące funkcje:

urllib.request.urlopen() - do pobrania zawartości strony
findAll() na obiekcie BeautifulSoup, można ją wykorzystać do przeiterowania po wszystkich tagach danego rodzaju
get_text() - Istnieje duża szansa, że wewnątrz tagów P znajdą się również inne tagi HTML, chcielibyśmy oczyścić 
z nich tekst. Można to zrobić albo z wyrażeniami regularnymi (robiliśmy takie zadanie na pierwszych laboratoriach!), albo użyć właśnie funkcji get_text() z BeautifulSoup

Linki do dokumentacji:
urllib, pobieranie danych: https://docs.python.org/3/howto/urllib2.html
beautifulSoup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/ (przeczytanie QuickStart jest wystarczające do zrobienia tego zadania)


In [31]:
import urllib.request
from bs4 import BeautifulSoup

with urllib.request.urlopen('http://www.cs.put.poznan.pl/swilk/pmwiki/pmwiki.php/Dydaktyka/InformatykaWMedycynie') as response:
   html = response.read()
soup = BeautifulSoup(html, 'html.parser')
urls = []
texts = []
for par in soup.find_all('p'):
    texts.append(par.get_text())
iterator = 0
for link in soup.find_all('a'):
    temp_link = link.get('href')
    if temp_link.startswith('http'):
        urls.append(temp_link)
        iterator += 1
    if iterator == 5:
        break

for url in urls:
    with urllib.request.urlopen(url) as response:
       new_html = response.read()
    new_soup = BeautifulSoup(new_html, 'html.parser')
    for par in new_soup.find_all('p'):
        texts.append(par.get_text())
print(texts)

['Poniżej zamieszczam slajdy z mojej części wykładowej:\n', 'Zajęcia 13.06 to drugi punkt kontrolny, na których powinny zostać zademonstrowane zrealizowane projekty. Obecność na tych zajęciach jest obowiązkowa. Każdą grupę projektową proszę o przygotowanie krótkiej (na ok. 15 minut) prezentacji zawierającej następujące elementy:\n', 'Poza prezentacją, każda grupa powinna też przygotować raport końcowy podsumowujący projekt. Poza elementami z prezentacji (w raporcie mogą być one -- a właściwie powinny być -- rozszerzone) powinny znaleźć się informacje dotyczące kompilacji, instalacji i uruchomienia projektu, które pozwolą na jego późniejsze samodzielne uruchomienie. Raport powinienem dostać do czwartku (16.06).\n', 'Zajęcia 9.05 to drugi punkt kontrolny, na którym poszczególne grupy prezentują aktualny stan zaawansowania prac. Przypominam, że w okresie między drugim a trzecim punktem kontrolnym każda grupa jest zobowiązana do pojawienia się na konsultacjach (jedna wizyta albo więcej).\n

# Zadanie 2 - CONLL
Dane ustrukturyzowane w formacie CONLL.

Niektóre algorytmy korzystają z dodatkowych metadanych opisujących poszczególne tokeny (słowa). Bardzo popularnym formatem zapisu takich danych jest format CONLL. 

Reprezentacja CONLL polega na tym, że dany tekst dzielony jest na zdania, a następnie każde zdanie dzielone jest na tokeny (tokenizowane). Następnie dla każdego tokenu tworzymy listę opisującą cechy tego tokenu (słowa).
Poniżej przykład wektora opisującego każdy token zadanego tekstu:
<ol>
    <li>ID - numer porządkowy tokenu w zdaniu</li>
    <li>text - tekst tokenu w formie nieprzetworzonej</li>
    <li>Part of Speech tag (POS tag) - informacja o części mowy, która powiązana jest z tym słowem </li>
    <li>is digit - flaga (o wartościach 0 lub 1), która informuje nas czy dany token jest liczbą</li>
    <li>is punct - flaga (o wartościach 0 lub 1), która informuje nas czy dany token jest znakiem interpunkcyjnym</li>
</ol>

Wektory dla kolejnych słów zapisywane są pod sobą. **Separatorem cech w wektorze jest pojedyncza spacja.**

**Zdania zwyczajowo oddzielamy od siebie podwójnym znakiem nowej linii.**

### Przykład:

Tekst: Kasia kupiła 2 lizaki: truskawkowy i zielony. Kasia używa Apple IPhone 5 i IPad.

Reprezentacja CONLL **(spacje separujące kolumny zostały zwielokrotnione na potrzeby zwiększenia czytelności)**
<pre>
1 Kasia  RZECZOWNIK 0 0
2 kupiła CZASOWNIK  0 0
3 2      LICZEBNIK  1 0
4 lizaki RZECZOWNIK 0 0
5 .      _          0 1

1 Kasia  RZECZOWNIK 0 0
2 używa  CZASOWNIK  0 0
3 Apple  RZECZOWNIK 0 0
4 IPhone RZECZOWNIK 0 0
5 5      LICZEBNIK  1 0
6 i      SPÓJNIK    0 0
7 iPad   RZECZOWNIK 0 0
8 .      _          0 1
</pre>

**Zadanie**: Napisz funkcję, która z zadanego tekstu w formie surowego tekstu stworzy reprezentację CONLL opisaną wcześniej wymienionymi atrybutami (ID, text, POS-tag, is_digit, is_punct).

Wykorzystaj sentence splitter i tokenizator z NLTK. Do uzyskania informacji o POS-tagach każdego tokenu wykorzystaj funkcję nltk.pos_tag(). W kolumnie związanej z POS-tagiem zapisz pos tag w takiej formie, w jakiej uzyskamy go z funkcji pos_tag (pos_tag() zwraca formy skrótowe, np. 'NN' dla rzeczowników).


In [61]:
from nltk import word_tokenize, sent_tokenize, pos_tag
def generate_conll(sentence):
    sents = sent_tokenize(sentence)
    for sent in sents:
        token_sent = word_tokenize(sent)
        pos_tags = pos_tag(token_sent)
        i = 0
        for tok in token_sent:
            is_digit = 0
            is_dot = 0
            if tok.isdigit():
                is_digit = 1
            if tok == ".":
                is_dot = 1
            if is_dot == 1:
                print(str(i) + str(tok) + " - " + " " + str(is_digit) + " " + str(is_dot))
            else:
                print(str(i) + " " + str(tok) + " "+ str(pos_tags[i][1]) + " " + str(is_digit) + " " + str(is_dot))
            i += 1
        print("")
generate_conll("Kate uses IPhone 5 and IPad. Kate bought 2 lolipops.")

0 Kate NNP 0 0
1 uses VBZ 0 0
2 IPhone NNP 0 0
3 5 CD 1 0
4 and CC 0 0
5 IPad NNP 0 0
6. -  0 1

0 Kate NNP 0 0
1 bought VBD 0 0
2 2 CD 1 0
3 lolipops NNS 0 0
4. -  0 1




Wyobraźmy sobie teraz, że chcielibyśmy wykrywać wzmianki o urządzeniach elektronicznych w tekście. W jaki sposób zakodować informację o (potencjalnie wielotokenowych) nazwach produktów w CONLL, tak, aby później móc wykonać proces uczenia?

Dodajmy w naszym CONLLu dodatkową kolumnę reprezentującą informację o urządzeniach elektronicznych.
Nazwy urządzeń mogą składać się potencjalnie z wielu słów.
Do zakodowania wielotokenowych tekstów używa się najczęściej notacji IOB, gdzie każda literka skrótu oznacza interpretację aktualnego słowa:
<ul>
    <li> B = begin, marker, który mówi, że aktualne słowo to początek nazwy </li>
    <li> I = inside, marker, który mówi, że aktualne słowo to kontynacja nazwy, która rozpoczyna się wystąpieniem wcześniejszego B</li>
    <li> O = outside, marker, który mówi, że aktualne słowo nie jest interesującą nas nazwą (jest poza nią) </li>
</ul>

Po dodaniu nowej kolumny (na końcu) nasz CONLL przybiera postać:

<pre>
1 Kasia  RZECZOWNIK 0 0 O
2 kupiła CZASOWNIK  0 0 O
3 2                 1 0 O
4 lizaki RZECZOWNIK 0 0 O
5 .      _          0 1 O

1 Kasia  RZECZOWNIK 0 0 O
2 używa             0 0 O
3 Apple  RZECZOWNIK 0 0 B
4 IPhone RZECZOWNIK 0 0 I
5 5                 1 0 I
6 i      SPÓJNIK    0 0 O
7 iPad   RZECZOWNIK 0 0 B
8 .      _          0 1 0
</pre>

**Zadanie**: Napisz funkcję, która wygeneruje CONLL z uwzględnieniem tagów IOB dotyczących urządzeń.
Nasza funkcja posiada teraz dodatkowy argument devices, który zawiera listę obiektów, które opisują gdzie (przesunięcie znakowe) znajduje się początek i koniec wzmianek.


In [67]:
def generate_CONLL_second(sentence, devices=[{"begin": 10, "end":18}, {"begin": 23, "end": 27}]):
    sents = sent_tokenize(sentence)
    for sent in sents:
        token_sent = word_tokenize(sent)
        pos_tags = pos_tag(token_sent)
        i = 0
        begin = -1
        for tok in token_sent:
            is_digit = 0
            is_dot = 0
            if tok.isdigit():
                is_digit = 1
            if tok == ".":
                is_dot = 1
            begin = sentence.find(tok)
            val_IOB = "O" 
            for d in devices:
                if begin == d["begin"]:
                    val_IOB = "B"
                if begin > d["begin"] and begin < d["end"]:
                    val_IOB = "I"
                    
            if is_dot == 1:
                print(i, tok, "-", is_digit, is_dot, val_IOB)
            else:
                print(i, tok, pos_tags[i][1], is_digit, is_dot, val_IOB)
            i += 1
        print("")
generate_CONLL_second("Kate uses IPhone 5 and IPad. Kate bought 2 lolipops.")

0 Kate NNP 0 0 O
1 uses VBZ 0 0 O
2 IPhone NNP 0 0 B
3 5 CD 1 0 I
4 and CC 0 0 O
5 IPad NNP 0 0 B
6 . - 0 1 O

0 Kate NNP 0 0 O
1 bought VBD 0 0 O
2 2 CD 1 0 O
3 lolipops NNS 0 0 O
4 . - 0 1 O

